<a href="https://colab.research.google.com/github/Jose897/7506-OrganizacionDeDatos/blob/main/Clase_1_Yo_Spark_BigMac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalamos e importamos librerías

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 64kB/s 
     |████████████████████████████████| 204kB 44.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=bf86b4da7effb373ceacc74cab5e2de39025659f46b51b89de1d351d52f64ac9
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos archivo con los datos del índice BigMac

In [4]:
downloaded = drive.CreateFile({'id':"1h9Lrqmc4uAkXeC0DJAg3f4VlJLut5C7S"})
downloaded.GetContentFile('bigmac.csv') 

# Creamos el Spark Context

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Leemos CSV

In [6]:
#Con header fuerzo a q tome la prmera como header(para indicarle q la primer fila son las columnas) y inferSchema(para q trate de inferir en los datos d las columnas, q detecte de q tipo son)
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('bigmac.csv', header=True, inferSchema=True)
rdd = df.rdd

## Cantidad de registros

In [7]:
#Cantidad de registros
rdd.count()

1730

## Qué datos tiene cada registro

In [23]:
#Vemos el primer registro
rdd.first()

Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2000-04-01')

In [8]:
#Acciones para ver algun registro
#Podemos usar la accion  .collect()   //Traemos todo, no es necesario
#Podemos usar la accion .first()      //para traer el primero
#Podemos usar la accon   take()       // para traer lo q quiera ()
rdd.take(5)

[Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2000-04-01'),
 Row(name='Australia', iso_a3='AUS', currency_code='AUD', local_price=2.59, dollar_ex=1.68, GDP_dollar=None, date='2000-04-01'),
 Row(name='Brazil', iso_a3='BRA', currency_code='BRL', local_price=2.95, dollar_ex=1.79, GDP_dollar=None, date='2000-04-01'),
 Row(name='Britain', iso_a3='GBR', currency_code='GBP', local_price=1.9, dollar_ex=0.632911392, GDP_dollar=None, date='2000-04-01'),
 Row(name='Canada', iso_a3='CAN', currency_code='CAD', local_price=2.85, dollar_ex=1.47, GDP_dollar=None, date='2000-04-01')]

## Qué país y cuando tuvo el mayor valor en USD

In [9]:
#Voy a sacar los q tengan dolares en 0 y el   ".cache()"   guarda el rdd con la accion ".filter" realizada
rdd = rdd.filter(lambda x: x.dollar_ex != 0).cache()

In [24]:
#Devolvemos el registro "x" if (peso/dolar > peso/dolar) else "y"    // devuelvo x si cumple y si no devuelvo y
rdd.reduce(lambda x,y: x if x.local_price / x.dollar_ex > y.local_price / y.dollar_ex else y)

Row(name='Venezuela', iso_a3='VEN', currency_code='VEF', local_price=39.0, dollar_ex=4.29465, GDP_dollar=None, date='2013-01-01')

In [25]:
#filtramos a "VEN"  y aplico la accion ".reduce"
rdd.filter(lambda x: x.iso_a3 != 'VEN').reduce(lambda x,y: x if x.local_price / x.dollar_ex > y.local_price / y.dollar_ex else y)

Row(name='Norway', iso_a3='NOR', currency_code='NOK', local_price=45.0, dollar_ex=5.41405, GDP_dollar=84443.634, date='2011-07-01')

## Y el menor?

In [26]:
#Realizo la misma accion mero on "<"
rdd.reduce(lambda x,y: x if x.local_price / x.dollar_ex < y.local_price / y.dollar_ex else y)

Row(name='Saudi Arabia', iso_a3='SAU', currency_code='SAR', local_price=2.4, dollar_ex=3.7502, GDP_dollar=None, date='2004-05-01')

## Precios de Argentina

In [27]:
#Vemos cuantos registros hay de ARGENTINA para ver si me alcanza en memoria
#34 registros esta bien para poder traerlos todos
rdd.filter(lambda x: x.iso_a3 == 'ARG').count()

34

In [13]:
#Filtro por Argentina y ".collect()" me traigo todos los registros
rdd.filter(lambda x: x.iso_a3 == 'ARG').collect()

[Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2000-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date='2001-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=3.13, GDP_dollar=None, date='2002-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.1, dollar_ex=2.88, GDP_dollar=None, date='2003-04-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.36, dollar_ex=2.95, GDP_dollar=None, date='2004-05-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.75, dollar_ex=2.897, GDP_dollar=None, date='2005-06-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=4.75, dollar_ex=3.0638, GDP_dollar=None, date='2006-01-01'),
 Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=7.0, dollar_ex=3.0565, GDP_dol

### En USD

In [14]:
#
arg = rdd.filter(lambda x: x.iso_a3 == 'ARG')

In [15]:
#Uso map clave-valor calve= fecha, valor (peso/dolar)
#Si uso el ".chache()" tengo q usarlo mas de 1 vez sino no tiene sentido usar el ".chache()"
arg_usd = arg.map(lambda x: (x.date, x.local_price/x.dollar_ex)).cache()

In [16]:
arg_usd.collect()

[('2000-04-01', 2.5),
 ('2001-04-01', 2.5),
 ('2002-04-01', 0.7987220447284346),
 ('2003-04-01', 1.423611111111111),
 ('2004-05-01', 1.4779661016949153),
 ('2005-06-01', 1.6396272005522956),
 ('2006-01-01', 1.550362295188981),
 ('2006-05-01', 2.290201210534925),
 ('2007-01-01', 2.670983407527317),
 ('2007-06-01', 2.668607472100922),
 ('2008-06-01', 3.642987249544627),
 ('2009-07-01', 3.0173827484421123),
 ('2010-01-01', 1.842832697117283),
 ('2010-07-01', 3.558945027009851),
 ('2011-07-01', 4.839685420447671),
 ('2012-01-01', 4.636606004404776),
 ('2012-07-01', 4.160963591568573),
 ('2013-01-01', 3.8179443383904355),
 ('2013-07-01', 3.8799076212471135),
 ('2014-01-01', 3.0341340075853354),
 ('2014-07-01', 2.5707727620504977),
 ('2015-01-01', 3.2520325203252036),
 ('2015-07-01', 3.0651340996168583),
 ('2016-01-01', 2.3897025544472004),
 ('2016-07-01', 3.3478406427854033),
 ('2017-01-01', 3.4683903515686585),
 ('2017-07-01', 4.1255341093266535),
 ('2018-01-01', 3.9603960396039604),
 ('20

### Mayor

In [17]:
#reutilizo gracias al ".chache()"
arg_usd.takeOrdered(3, lambda x: -x[1])

[('2011-07-01', 4.839685420447671),
 ('2012-01-01', 4.636606004404776),
 ('2012-07-01', 4.160963591568573)]

### Menor

In [18]:
#reutilizo gracias al ".chache()"
arg_usd.takeOrdered(3, lambda x: x[1])

[('2002-04-01', 0.7987220447284346),
 ('2003-04-01', 1.423611111111111),
 ('2004-05-01', 1.4779661016949153)]

### 5 mayores

### 5 menores

### Valor promedio

In [19]:
arg_usd.map(lambda x: x[1]).reduce(lambda x,y: x+y) / arg_usd.count()

2.9356961998694073

## Valores en USA

In [20]:
rdd.filter(lambda x: x.iso_a3 == 'USA').map(lambda x: (x.date, x.local_price)).collect()

[('2000-04-01', 2.51),
 ('2001-04-01', 2.54),
 ('2002-04-01', 2.49),
 ('2003-04-01', 2.71),
 ('2004-05-01', 2.9),
 ('2005-06-01', 3.06),
 ('2006-01-01', 3.15),
 ('2006-05-01', 3.1),
 ('2007-01-01', 3.22),
 ('2007-06-01', 3.41),
 ('2008-06-01', 3.57),
 ('2009-07-01', 3.57),
 ('2010-01-01', 3.58),
 ('2010-07-01', 3.733333333),
 ('2011-07-01', 4.065),
 ('2012-01-01', 4.19722),
 ('2012-07-01', 4.3275),
 ('2013-01-01', 4.367395833),
 ('2013-07-01', 4.556666667),
 ('2014-01-01', 4.624166667),
 ('2014-07-01', 4.795),
 ('2015-01-01', 4.79),
 ('2015-07-01', 4.79),
 ('2016-01-01', 4.93),
 ('2016-07-01', 5.04),
 ('2017-01-01', 5.06),
 ('2017-07-01', 5.3),
 ('2018-01-01', 5.28),
 ('2018-07-01', 5.51),
 ('2019-01-01', 5.58),
 ('2019-07-09', 5.74),
 ('2020-01-14', 5.67),
 ('2020-07-01', 5.71),
 ('2021-01-01', 5.66)]

## 10 más caros en promedio durante los 20 años

In [28]:
#el "\" es para continuar a bajo el
#map clave velor,valor   //ES COSTOSO EL MAP EN SPARK
# map         // transforma "name" como clave y valor serian las columnas (importe,1)
# reduceByKey // pasa a tomar como clave la clave de map y pasaria a columna 0 el de importe;
#             //suma importes de la columna 0 y la cantidad de valores q tengo en la columna 1
# map         //reacomoda la clave y valor, clave-columna x[0]= name y valor-columna x[1]= dupla_columna(contiene importe y cantidad de registros)
#             // -> X[1] dupla_columna / x[1][0] el importe y x[1][1] es la cantidad de registros
#.takeOdered()//devuelve los 10, "-" mayores de la columna x[1] serian los importe en porcentaje
rdd.map(lambda x: (x.name, (x.local_price / x.dollar_ex, 1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
   .map(lambda x: (x[0],x[1][0]/x[1][1]))\
   .takeOrdered(10, lambda x:-x[1])

[('Norway', 6.380225767510909),
 ('Switzerland', 6.1111973283220085),
 ('Sweden', 5.21455311225687),
 ('Finland', 5.1629535999708125),
 ('Italy', 4.7913481249763565),
 ('France', 4.76472499997649),
 ('Belgium', 4.749227374984401),
 ('Denmark', 4.714949725134277),
 ('Ireland', 4.640071099992816),
 ('Germany', 4.599525249984549)]

In [29]:
#el "\" es para continuar a bajo el
#map clave velor,valor   //ES COSTOSO EL MAP EN SPARK
# map         // transforma "name" como clave y valor serian las columnas (importe,1)
# reduceByKey // pasa a tomar como clave la clave de map y pasaria a columna 0 el de importe;
#             //suma importes de la columna 0 y la cantidad de valores q tengo en la columna 1


#.takeOdered()//devuelve los 10, "-" mayores de la columna x[1] serian los importe en porcentaje
rdd.map(lambda x: (x.name, (x.local_price / x.dollar_ex, 1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
   .takeOrdered(10, lambda x:-x[1][0]/x[1][1])

[('Norway', (191.40677302532728, 30)),
 ('Switzerland', (207.78070916294828, 34)),
 ('Sweden', (177.29480581673357, 34)),
 ('Finland', (103.25907199941625, 20)),
 ('Italy', (95.82696249952713, 20)),
 ('France', (95.2944999995298, 20)),
 ('Belgium', (94.98454749968802, 20)),
 ('Denmark', (160.3082906545654, 34)),
 ('Ireland', (92.80142199985632, 20)),
 ('Germany', (91.99050499969098, 20))]

## 10 más baratos